In [9]:
from bs4 import BeautifulSoup, Comment
from datetime import datetime, timedelta, timezone

import requests
import pandas as pd
import time

In [25]:
dates = pd.date_range(start = "2024-01-01", end = "2024-05-09")
base_link = "https://www.basketball-reference.com"

for date in dates:
    link = base_link + "/boxscores/?month={0}&day={1}&year={2}".format(date.month, date.day, date.year)

    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")

    # Get all games for the day
    try:
        games = soup.find("div", class_ = "game_summaries").find_all("table", class_= "teams")
        # Loop through all games to get boxscore link
        for game in games:
            boxscore_link = game.find("td", class_ = "gamelink").find("a", href = True)['href']

            # Access boxscore
            page = requests.get(base_link + boxscore_link)
            print(boxscore_link)
            time.sleep(2)
            soup = BeautifulSoup(page.content, "html.parser")

            teams = soup.find("div", class_="scorebox").find_all("strong")
            team1_text = teams[0].text.strip()
            team2_text = teams[1].text.strip()
            
            #team1 = Team.objects.get(team_abbr = team1_text)
            #team2 = Team.objects.get(team_abbr = team2_text)

            # Line Score data is stored in comments so this is a work around
            comment = soup.find("div", id = 'all_line_score').find(string=lambda text: isinstance(text, Comment))
            if "div_line_score" in comment:
                line_score = BeautifulSoup(comment.string, "html.parser")

                data = line_score.find('tbody').find_all("td")
                team1_q1_points = int(data[0].text.strip())
                team1_q2_points = int(data[1].text.strip())
                team1_q3_points = int(data[2].text.strip())
                team1_q4_points = int(data[3].text.strip())

                team2_q1_points = int(data[4].text.strip())
                team2_q2_points = int(data[5].text.strip())
                team2_q3_points = int(data[6].text.strip())
                team2_q4_points = int(data[7].text.strip())

            # Four Factors data is stored in comments
            comment = soup.find("div", id = 'all_four_factors').find(string=lambda text: isinstance(text, Comment))
            if "div_four_factors" in comment:
                factors = BeautifulSoup(comment.string, "html.parser")
                
                data = factors.find_all('td')
                team1_pace = float(data[0].text.strip())
                team1_efg = float(data[1].text.strip())
                team1_tov = float(data[2].text.strip())
                team1_orb = float(data[3].text.strip())
                team1_ft_per_fg = float(data[4].text.strip())
                team1_off_rtg = float(data[5].text.strip())

                team2_pace = float(data[6].text.strip())
                team2_efg = float(data[7].text.strip())
                team2_tov = float(data[8].text.strip())
                team2_orb = float(data[9].text.strip())
                team2_ft_per_fg = float(data[10].text.strip())
                team2_off_rtg = float(data[11].text.strip())   
            
            # box_score = BoxScore(date=date.strftime('%Y-%m-%d'),
            #                         team1 = team1,
            #                         team2 = team2,
            #                         team1_q1_points = team1_q1_points,
            #                         team1_q2_points = team1_q2_points,
            #                         team1_q3_points = team1_q3_points,
            #                         team1_q4_points = team1_q4_points,
            #                         team2_q1_points = team2_q1_points,
            #                         team2_q2_points = team2_q2_points,
            #                         team2_q3_points = team2_q3_points,
            #                         team2_q4_points = team2_q4_points,
            #                         team1_pace = team1_pace,
            #                         team1_efg = team1_efg,
            #                         team1_tov = team1_tov,
            #                         team1_orb = team1_orb,
            #                         team1_ft_per_fg = team1_ft_per_fg,
            #                         team1_off_rtg = team1_off_rtg,
            #                         team2_pace = team2_pace,
            #                         team2_efg = team2_efg,
            #                         team2_tov = team2_tov,
            #                         team2_orb = team2_orb,
            #                         team2_ft_per_fg = team2_ft_per_fg,
            #                         team2_off_rtg = team2_off_rtg)
            # box_score.save()
            print(f"BOX SCORE FOR {team1_text} vs {team2_text} ON {date.month}-{date.day}-{date.year} ADDED!")
    except Exception as e:
        print(f"NO GAMES PLAYED ON {date.month}-{date.day}-{date.year}.")
    

NO GAMES PLAYED ON 7-1-2024.
NO GAMES PLAYED ON 7-2-2024.


In [47]:
today = datetime.today()
tomorrow = today + timedelta(days=1)

In [48]:
today = today.strftime("%Y-%m-%dT00:00:00Z")
tomorrow = tomorrow.strftime("%Y-%m-%dT04:00:00Z")
print(today, tomorrow)

2024-05-09T00:00:00Z 2024-05-10T04:00:00Z


In [19]:
resp = requests.get("https://api.the-odds-api.com/v4/sports/basketball_nba/odds",
                    params={
                        "api_key" : "18d4f5a57823e4304f11070456e9b30e",
                        "regions" : "us",
                        "markets" : "h2h,spreads,totals",
                        "oddsFormat" : "american",
                        "dateFormat" : "iso",
                        "bookmakers" : "fanduel",
                        "commenceTimeFrom" : str(today),
                        "commenceTimeTo" : tomorrow
                    })

In [20]:
if resp.status_code != 200:
    print(f'Failed to get odds: status_code {resp.status_code}, response body {resp.text}')

else:
    odds_json = resp.json()
    #print('Number of events:', len(odds_json))
    # Check the usage quota
    #print('Remaining requests', resp.headers['x-requests-remaining'])
    #print('Used requests', resp.headers['x-requests-used'])

Number of events: 2
[{'id': 'e395974295e16027bfeaf8e1d776ddee', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2024-05-09T23:00:00Z', 'home_team': 'Boston Celtics', 'away_team': 'Cleveland Cavaliers', 'bookmakers': [{'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2024-05-09T16:18:25Z', 'markets': [{'key': 'h2h', 'last_update': '2024-05-09T16:18:25Z', 'outcomes': [{'name': 'Boston Celtics', 'price': -950}, {'name': 'Cleveland Cavaliers', 'price': 640}]}, {'key': 'spreads', 'last_update': '2024-05-09T16:18:25Z', 'outcomes': [{'name': 'Boston Celtics', 'price': -112, 'point': -13.5}, {'name': 'Cleveland Cavaliers', 'price': -108, 'point': 13.5}]}, {'key': 'totals', 'last_update': '2024-05-09T16:18:25Z', 'outcomes': [{'name': 'Over', 'price': -110, 'point': 212.5}, {'name': 'Under', 'price': -110, 'point': 212.5}]}]}]}, {'id': '56493f78ffa4a7745c6c011bcc635091', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2024-05-10T01:30:00Z', 'hom

In [24]:
resp.text

'[{"id":"e395974295e16027bfeaf8e1d776ddee","sport_key":"basketball_nba","sport_title":"NBA","commence_time":"2024-05-09T23:00:00Z","home_team":"Boston Celtics","away_team":"Cleveland Cavaliers","bookmakers":[{"key":"fanduel","title":"FanDuel","last_update":"2024-05-09T16:18:25Z","markets":[{"key":"h2h","last_update":"2024-05-09T16:18:25Z","outcomes":[{"name":"Boston Celtics","price":-950},{"name":"Cleveland Cavaliers","price":640}]},{"key":"spreads","last_update":"2024-05-09T16:18:25Z","outcomes":[{"name":"Boston Celtics","price":-112,"point":-13.5},{"name":"Cleveland Cavaliers","price":-108,"point":13.5}]},{"key":"totals","last_update":"2024-05-09T16:18:25Z","outcomes":[{"name":"Over","price":-110,"point":212.5},{"name":"Under","price":-110,"point":212.5}]}]}]},{"id":"56493f78ffa4a7745c6c011bcc635091","sport_key":"basketball_nba","sport_title":"NBA","commence_time":"2024-05-10T01:30:00Z","home_team":"Oklahoma City Thunder","away_team":"Dallas Mavericks","bookmakers":[{"key":"fanduel",

In [62]:
est_offset = timedelta(hours=-6)
original_timezone = timezone.utc

In [76]:
for json in odds_json:
    home_team = json['home_team']
    away_team = json['away_team']
    time = datetime.fromisoformat(json['commence_time']).astimezone(original_timezone) + est_offset
    bm = json['bookmakers']

    # Will need to change if more sports books are added
    bookmaker = bm[0]['title']
    markets = bm[0]['markets']

    for m in markets:
        key = m['key']
        for team in m['outcomes']:
            if team['name'] == home_team:
                if key == 'h2h':
                    home_h2h_price = team['price']
                elif key == 'spreads':
                    home_spreads_price = team['price']
                    home_spreads_points = team['point']
            elif team['name'] == away_team:
                if key == 'h2h':
                    away_h2h_price = team['price']
                elif key == 'spreads':
                    away_spreads_price = team['price']
                    away_spreads_points = team['point']
            elif team['name'] == "Over":
                over_price = team['price']
                over_point = team['point']
            else:
                under_price = team['price']
                under_point = team['point']


    print(time)
    print(home_team, home_h2h_price, home_spreads_price, home_spreads_points)
    print(away_team, away_h2h_price, away_spreads_price, away_spreads_points)
    print(over_price, over_point)
    print(under_price, under_point)
    print()



2024-05-09 17:00:00+00:00
Boston Celtics -950 -112 -13.5
Cleveland Cavaliers 640 -108 13.5
-110 212.5
-110 212.5

2024-05-09 19:30:00+00:00
Oklahoma City Thunder -200 -110 -5.0
Dallas Mavericks 168 -110 5.0
-108 218.0
-112 218.0



In [22]:
import json

print(json.dumps(odds_json, indent=4))

[
    {
        "id": "e395974295e16027bfeaf8e1d776ddee",
        "sport_key": "basketball_nba",
        "sport_title": "NBA",
        "commence_time": "2024-05-09T23:00:00Z",
        "home_team": "Boston Celtics",
        "away_team": "Cleveland Cavaliers",
        "bookmakers": [
            {
                "key": "fanduel",
                "title": "FanDuel",
                "last_update": "2024-05-09T16:18:25Z",
                "markets": [
                    {
                        "key": "h2h",
                        "last_update": "2024-05-09T16:18:25Z",
                        "outcomes": [
                            {
                                "name": "Boston Celtics",
                                "price": -950
                            },
                            {
                                "name": "Cleveland Cavaliers",
                                "price": 640
                            }
                        ]
                    },
      